<h1>HVAC Analysis of Ball Arena</h1>
This notebook looks into the requirements for real time opearion of HVAC systems in Ball Arena (Denver, Co.), as well as some demonstration metrics.

<h2>Setup</h2>
Load dependencies for project:

In [7]:
import matplotlib
import psychrolib
import pyromat as pm
import ht 
from pyfluids import Fluid, FluidsList, Input
import math

<h2>Define Useful Functions</h2>
These funcitons include converting °F to °C, °C to K, and converting between m and ft.

In [8]:
def getCelciusFromFahrenheit(tf):
    return (tf - 32) / 1.8

def getSquMeterFromSquFt(squFt):
    return squFt * 0.092903

def celciusToKelvin(cel):
    return cel + 273.15

def getMeterFromFeet(feet):
    return feet * 0.3048

<h2> Define Psychrometric Functions </h2>
These functions will be used throughout the code to get psychromtric data pertienent to modeling HVAC systems.

In [9]:
#For retriving water and air variables 
h2o = pm.get("ig.H2O")
air = pm.get("ig.air")

# For calculating P_sat given temperature| temperature in K, out in kPa
def getPsat(temperature):
    return h2o.p(T=temperature)

# For determining current water pressure given temperature in K, relativeHumidity in %, out in kPa
def getWaterPressure(temperature, relativeHumidity):
    return relativeHumidity / 100 * getPsat(temperature)

#Define R value for air [IG model]
Ra = 8.314

# for getting mass flow rate of water| temperature in K, relativeHumidity in %, volumeFlow in m^3/s, out in kg/s 
def getWaterMass(temperature, relativeHumidity, volumeFlow):
    return getWaterPressure(temperature, relativeHumidity) * volumeFlow * h2o.mw() / Ra / temperature

# for getting mass flow rate| temperature in K, dryAirPressure in kPa, volumeFlow in m^3/s, out in kg/s 
def getDryAirMass(temperature, dryAirPressure, volumeFlow):
    return dryAirPressure * volumeFlow * h2o.mw() / Ra / temperature

# Used for getting the dry air pressure| pressure in kPa, waterPressure in kPa, out in kPa
def getDryAirPressure(pressure, waterPressure):
    return pressure - waterPressure

# Used for getting the specific humidity| pressure in kPa, waterPressure in kPa, out in 0 to 1
def getSpecificHumidity(pressure, waterPressure):
    return 0.622 * waterPressure / (pressure - waterPressure)


# For getting water enthalpy| temperature in K, pressure in kPa, out in kJ/kmol
def getWaterEnthalpy(temperature, pressure):
    return h2o.h(T=temperature, p=pressure)

# For getting dry air enthalpy| temperature in K, pressure in kPa, out in kJ/kmol
def getDryAirEnthalpy(temperature, pressure):
    return air.h(T=temperature, p=pressure)
    
# For getting water enthalpy flow| temperature in K, pressure in kPa, massFlow in kg/s, out in kJ/s
def getWaterEnthalpyFlow(temperature, pressure, massFlow):
    return getWaterEnthalpy(temperature, pressure) / h2o.mw() * massFlow

# For getting dry air enthalpy flow| temperature in K, pressure in kPa, massFlow in kg/s, out in kJ/s
def getDryAirEnthalpyFlow(temperature, pressure, massFlow):
    return getDryAirEnthalpy(temperature, pressure) / air.mw() * massFlow
    
# For determining required refrigerant| qout in kW, out in tons of refrigerant
def getRequiredRefrigerant(qout):
    return .2893 * qout

def getDryAirMassFlowRate(pressure, vaporpressure, volumeflowrate, temperature):
    return volumeflowrate / (air.v(T=temperature) + getSpecificHumidity(pressure, vaporpressure) * h2o.v(T=temperature))

def getVaporMassFlowRate(pressure, vaporpressure, volumeflowrate, temperature):
    return volumeflowrate / (air.v(T=temperature) / getSpecificHumidity(pressure, vaporpressure) + h2o.v(T=temperature))

<h2> Define Arena Variables </h2>
Data related to the arena and game requirements

In [10]:
# Stadium size information
stadiumFloorArea = getSquMeterFromSquFt(675000) # m^2
stadiumSizeRatio = 85/200
stadiumLength = math.sqrt(stadiumFloorArea / (1 + stadiumSizeRatio)) # m
stadiumWidth = stadiumFloorArea / stadiumLength # m
arenaWidth = getMeterFromFeet(85 * 10 / 1.5)
arenaLength = getMeterFromFeet(200 * 12.5 / 2.5)
stadiumHeight = 15 # m (Guess for now cant find any actual data)

print("Stadium Floor Area: " + str(stadiumFloorArea) + "m^2 , Stadium Length: " + str(stadiumLength) + "m , Stadium Width: " + str(stadiumWidth) + "m")

Stadium Floor Area: 62709.525m^2 , Stadium Length: 209.77770189065927m , Stadium Width: 298.9332251941895m


<h2> Desired Game Conditions <h2>

In [ ]:
# Values related to the game
gameLength = 2.5 # hours

# Desired conditions to be mainted throughout the game
desiredHumidity = 35 # %
desiredTemperature = getCelciusFromFahrenheit(63) # C
airPressure = 84.0 # Kpa

$\hat{Y} = \hat{\beta}_{0} + \sum \limits _{j=1} ^{p} X_{j}\hat{\beta}_{j} $